# Flamingo

## 参考

- [Flamingo: a Visual Language Model for Few-Shot Learning][0]
- [OpenFlamingo: An Open-SOurce Framework for Training Large Autogressive Vision-Language Models][1]
- [mlfoundations / open_flamingo][2]

[0]: https://arxiv.org/abs/2204.14198
[1]: https://arxiv.org/abs/2308.01390
[2]: https://github.com/mlfoundations/open_flamingo

## 概要

Flamingoは、少数のアノテーションサンプルで新しいタスクに迅速に対応できる視覚言語モデル（Visual Language Model, VLM）

事前学習済みの視覚モデルと言語モデルを連携し、画像・動画とテキストの入力に対応したアーキテクチャを採用

インターネットから収集した大規模なテキストと画像が混在するコーパスで訓練

コンテキストに回答例を含めて新しいタスクに柔軟に対応する学習を実現（in-context few-shot learning）

訓練したモデルを詳細に評価:

1. 画像に関する質問に回答するオープンエンドなタスク（Visual Question-Answering, VQA）
2. 画像や動画にキャプションをつけるタスク
3. 画像に関する多肢選択式の質問に回答するクローズドエンドなタスク

多くのベンチマークで、ファインチューニング済みの他のモデルよりも高い性能

Flaming-80Bでのフューショットプロンプトに対する出力サンプル:

![](image/fig1_1.png)

複数の画像にわたる対話も可能:

![](image/fig1_2.png)

Flamingoが16のうち6つのタスクで最先端の特化モデルよりも優れていて、モデルサイズと例示数が増えると性能が上がる:

![](image/fig2.png)

## アーキテクチャ

Flamingoは、2つのパラメータを固定した（frozen）モデルを活用:

1. 視覚モデル（Vision Encoder）: 画像や動画を知覚する
2. 言語モデル（LM block）: 基本的な推論を行う

事前学習中に獲得した知識を保持するコンポーネント:

- Perceiver Resampler: 可変な高解像度の画像や動画を、少数の固定された視覚トークンに変換する
- ゲート付きクロスアテンション密結合（GATED XATTN-DENSE）: 視覚モデルと言語モデルを橋渡しする

![](image/fig3.png)

Flamingoは、生成するテキスト全体 $y$ の尤度を次式でモデル化:

$$
p(y|x) = \prod_{l=1}^L p(y_l | y_{\le l}, x_{\le l})
$$

- テキスト全体の生成確率 $p(y|x)$ は、今までに見た全ての視覚トークン $x_{\le l}$ と今までに生成したテキストトークン $y_{\le l}$ の条件下で次のトークン $y_l$ が生成される確率を、最初から最後までかけ合わせた値
- $y_l$: テキスト $y$ の $l$ 番目の言語トークン
- $y_{\le l}$: $l$ 番目のトークンより前にある全ての言語トークン
- $x_{\le l}$: $l$ 番目のトークンが処理されるより前のすべての画像や動画の集合
- $p(\cdot)$: モデルによって推論される確率

視覚エンコーダーの特徴:

- Normalizer-Free ResNet（NFNet）を使用
- CLIPと同様の対象学習損失（two-term contrastive loss）で事前学習
- 視覚エンコーダーの出力は1次元にフラット化される
- 動画入力の場合、1FPSでサンプリングし、独立してエンコードし、学習済みの時間埋め込みを加算しフラット化
- NFNetエンコーダーのようなCNNの場合、暗黙的に空間情報をチャンネルごとに含んでいるため画像の位置埋め込みは必要ない

Perceiver Resamplerの特徴:

- 視覚エンコーダーからの可変長な入力を、固定長（=64トークン）の視覚トークン（latent input queries）に変換
- Perceiver Resamplerは、学習可能な潜在クエリ（=64トークン）を入力としたクロスアテンションを計算する複数のTransformerで構成される

![](image/fig5.png)

ゲート付きクロスアテンド密結合（GATED XATTN-DENSE）の特徴：

- ゲート付きのクロスアテンション層とゲート付きフィードフォワード層で構成される
- 事前学習済みの言語モデルブロックを凍結し、元の層の間にGATED XATTN-DENSEを追加し、ゼロから訓練する
- ゲートはtanh-gating（ハイパボリックタンジェントゲーティング）を採用し、ゼロで初期化
- 訓練の最初は元の言語モデルと変わらないため学習が安定化し、その結果最終的な結果も改善

![](image/fig4.png)

言語モデルはChinchillaモデルを使用し、3つのサイズで実験:

- 1.4B（Flamingo-3B）
- 7B（Flamingo-9B）
- 70B（Flamingo-80B）

画像因果モデリング（image-causal modeling）の仕組み:

- 画像因果モデリングは、今まで出現したテキストと画像でトークンの生成が条件付けられる性質
- クロスアテンション行列にマスキングし、アテンドできる視覚トークンを「直前に出現した画像のみ」に制限することで実現
- この単一画像クロスアテンション（single-image cross attention）により、複数枚画像入力に汎化可能になる

訓練は3種類のデータセットの混ぜ合わせて実施:

1. 画像とテキストが交互に配置されたサンプルで構成されるデータセット
    - 4300万件のウェブページから抽出し、HTML構造をもとに再構成したM3W（MultiModal Massive Web）を作成
    - 画像の位置に`<image>`タグを追加し、各画像の直前とドキュメント末尾に`<EOC>`（end of chunk）トークンを追加
    - ドキュメントごとに $L=256$ トークンをランダムにサンプリングし、$N = 5$ 枚までの画像を使用
2. 画像とテキストのペアで構成されるデータセット
    - 18億の画像とaltテキストがペアになったALIGNデータセットをベースに使用
    - 更により高品質で長い説明文を含む3億1200万件のデータセットを作成し補完（LTIP, Long Text & Image Pairs）
3. 動画とテキストのペアで構成されるデータセット
    - 平均約22秒の短い動画と説明文がペアになった2700万件のデータで構成（VTP, Video & Text Pairs）

訓練の損失関数は、データセットごとに重み付けされた期待負対数尤度（expected negative log-liklihood）を使用:

$$
\sum_{m=1}^M \lambda_m \cdot \mathbb{E}_{(x,y) \sim D_m} \left[ - \sum_{l=1}^L \log p(y_l | y_{\lt l}, x_{\le l}) \right]
$$

- 各データセット（$D_m$）について、負の対数尤度の平均（$\mathbb{E}$）を計算し、各データセットの重要度 $\lambda_m$ で重み付けし合計した値
- $D_m$: $m$ 番目のデータセット
- $\lambda_m$: $m$ 番目のデータセットの重要度

訓練では、全てのデータセットからバッチを計算し、それぞれの勾配を個別に計算し、合算してパラメータを更新する（多目的訓練）

訓練後は、インコンテキスト学習（in-context learning）を用いてモデルのタスク適応力を評価:

- 画像とテキストがペアになった回答例（support example pairs）の後に、クエリとなる画像を追加してプロンプトを構成
- オープンエンドの評価では、ビームサーチを使用
- クローズドエンドの評価では、モデルの対数尤度を用いて選択肢をスコアリング

## 実験

11のベンチマークで性能を検証し、そのうち7つが画像と動画認識タスクで特化モデルより優れた性能を示した:

![](image/table1.png)

FlamingoがSotAに達しなかった13のベンチマークは、入力解像度を上げるファインチューニングをすると5つが改善:

![](image/table2.png)

Flamingo-3Bでアブレーション研究をしたところ訓練データの混合（M3W）が特に重要であることがわかった:

![](image/table3.png)

## 実装

[Open-Flamingo][1]を参考にして実装する。

[1]: https://github.com/mlfoundations/open_flamingo

### 環境構築

In [ ]:
# ログ設定

import logging
import os

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)
logger.info("ログを初期化")

In [ ]:
%pip install -q open_clip_torch einops einops_exts

In [ ]:
from einops import rearrange
from einops import repeat
from einops_exts import rearrange_many
from huggingface_hub import hf_hub_download
from PIL import Image
from torch import einsum, nn
from torch import nn
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp.wrap import enable_wrap, wrap
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast
from typing import Optional
import open_clip
import requests
import torch
import torch.nn as nn

### ヘルパー関数

In [ ]:
def exists(val):
    """
    値がNoneでないかを検証する

    引数:
        val: 検証する値
    戻り値:
        bool: 値がNoneでない場合にTrueを返す
    """
    return val is not None

exists("test")

In [ ]:
def getattr_recursive(obj, att):
    """
    オブジェクトのネストされた属性を属性名で取得する

    引数:
        obj: 属性を取得するオブジェクト
        att (str): ネストされた属性名（ドットで区切られた文字列）
    戻り値:
        属性の値
    """
    if att == "":
        return obj
    i = att.find(".")
    if i < 0:
        return getattr(obj, att)
    else:
        return getattr_recursive(getattr(obj, att[:i]), att[i + 1 :])

# 検証

class Nested:
    b = None

class Sample:
    a = Nested()

sample = Sample()
sample.a.b = 42
getattr_recursive(sample, "a.b")

In [ ]:
def setattr_recursive(obj, att, val):
    """
    オブジェクトにネストされた属性を属性名で設定する

    引数:
        obj: 属性を設定するオブジェクト
        att (str): ネストされた属性名（ドットで区切られた文字列）
        val: 設定する値
    """
    if "." in att:
        obj = getattr_recursive(obj, ".".join(att.split(".")[:-1]))
    setattr(obj, att.split(".")[-1], val)

# 検証

sample = Sample()
setattr_recursive(sample, "a.b", 42)
sample.a.b

### FeedFoward

In [ ]:
def FeedForward(dim, mult=4):
    """
    フィードフォワードニューラルネットワークを作成する
    PerceiverResamplerとGatedCrossAttentionで使用する

    引数:
        dim (int): 入力および出力の次元数
        mult (int, optional): 内部層の次元数を決定するための乗数。デフォルトは4。
    戻り値:
        nn.Sequential: フィードフォワードニューラルネットワークのシーケンシャルモデル
    """
    inner_dim = int(dim * mult)

    return nn.Sequential(
        nn.LayerNorm(dim),
        nn.Linear(dim, inner_dim, bias=False),
        nn.GELU(),
        nn.Linear(inner_dim, dim, bias=False),
    )

ff = FeedForward(128, mult=2)
ff

### MaskedCrossAttention

In [ ]:
class MaskedCrossAttention(nn.Module):
    """
    マスク付きクロスアテンションモジュール
    GatedCrossAttentionBlockで使用
    """

    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        only_attend_immediate_media=True,
    ):
        logger.info(f"MaskedCrossAttentionを初期化 {dim=}, {dim_visual=}, {dim_head=}, {heads=}, {only_attend_immediate_media=}")

        super().__init__()

        self.scale = dim_head**-0.5
        logger.info(f"{self.scale=}")

        self.heads = heads

        inner_dim = dim_head * heads
        logger.info(f"{inner_dim=}")

        self.norm = nn.LayerNorm(dim)

        self.to_q = nn.Linear(dim, inner_dim, bias=False)

        self.to_kv = nn.Linear(dim_visual, inner_dim * 2, bias=False)

        self.to_out = nn.Linear(inner_dim, dim, bias=False)

        # whether for text to only attend to immediate preceding image, or all previous images
        self.only_attend_immediate_media = only_attend_immediate_media

        logger.info("MaskedCrossAttentionの初期化完了")


    def forward(self, x, media, media_locations=None, use_cached_media=False):
        """
        マスク付きクロスアテンションの順伝播

        引数:
            x (torch.Tensor): テキスト特徴量
                shape (B, T_txt, D_txt)
            media (torch.Tensor): 画像特徴量
                shape (B, T_img, n, D_img) ここでnは潜在変数の次元
            media_locations: x内のメディアトークンを識別するブールマスク
                shape (B, T_txt)
            use_cached_media: bool
                Trueの場合、media_locationsで登録された最後のメディア以降のすべてのxを扱う。T_txtはmedia_locations.shape[1]と正確に等しくある必要はない
        戻り値:
            torch.Tensor: クロスアテンションの出力
        """
        logger.info(f"MaskedCrossAttentionの順伝播開始 {x.shape=}, {media.shape=}, {None if media_locations is None else media_locations.shape=}, {use_cached_media=}")

        if not use_cached_media:
            assert (
                media_locations.shape[1] == x.shape[1]
            ), f"media_location.shape is {media_locations.shape} but x.shape is {x.shape}"

        T_txt = x.shape[1]

        _, T_img, n = media.shape[:3]

        h = self.heads

        x = self.norm(x)

        q = self.to_q(x)

        media = rearrange(media, "b t n d -> b (t n) d")

        k, v = self.to_kv(media).chunk(2, dim=-1)
        q, k, v = rearrange_many((q, k, v), "b n (h d) -> b h n d", h=h)

        q = q * self.scale

        sim = einsum("... i d, ... j d -> ... i j", q, k)

        if exists(media_locations):
            media_time = torch.arange(T_img, device=x.device) + 1

            if use_cached_media:
                # text time is set to the last cached media location
                text_time = repeat(
                    torch.count_nonzero(media_locations, dim=1),
                    "b -> b i",
                    i=T_txt,
                )
            else:
                # at each boolean of True, increment the time counter (relative to media time)
                text_time = media_locations.cumsum(dim=-1)

            # text time must equal media time if only attending to most immediate image
            # otherwise, as long as text time is greater than media time (if attending to all previous images / media)
            mask_op = torch.eq if self.only_attend_immediate_media else torch.ge

            text_to_media_mask = mask_op(
                rearrange(text_time, "b i -> b 1 i 1"),
                repeat(media_time, "j -> 1 1 1 (j n)", n=n),
            )
            sim = sim.masked_fill(~text_to_media_mask, -torch.finfo(sim.dtype).max)

        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        if exists(media_locations) and self.only_attend_immediate_media:
            # any text without a preceding media needs to have attention zeroed out
            text_without_media_mask = text_time == 0
            text_without_media_mask = rearrange(
                text_without_media_mask, "b i -> b 1 i 1"
            )
            attn = attn.masked_fill(text_without_media_mask, 0.0)

        out = einsum("... i j, ... j d -> ... i d", attn, v)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.to_out(out)

        logger.info(f"MaskedCrossAttentionの順伝播完了 {out.shape=}")
        return out

### GatedCrossAttentionBlock

In [ ]:
class GatedCrossAttentionBlock(nn.Module):
    """
    ゲート付きクロスアテンションブロック
    FlamingoLayerとFlamingoLMMixinで使用
    """

    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        ff_mult=4,
        only_attend_immediate_media=True,
    ):
        logger.info(f"GatedCrossAttentionBlockを初期化 {dim=}, {dim_visual=}, {dim_head=}, {heads=}, {ff_mult=}, {only_attend_immediate_media=}")

        super().__init__()

        self.attn = MaskedCrossAttention(
            dim=dim,
            dim_visual=dim_visual,
            dim_head=dim_head,
            heads=heads,
            only_attend_immediate_media=only_attend_immediate_media,
        )
        self.attn_gate = nn.Parameter(torch.tensor([0.0]))

        self.ff = FeedForward(dim, mult=ff_mult)
        self.ff_gate = nn.Parameter(torch.tensor([0.0]))
        logger.info("GatedCrossAttentionBlockの初期化完了")

    def forward(
        self,
        x,
        media,
        media_locations=None,
        use_cached_media=False,
    ):
        """
        ゲート付きクロスアテンションブロックの順伝播

        引数:
            x (torch.Tensor): テキスト特徴量
                shape (B, T_txt, D_txt)
            media (torch.Tensor): 画像特徴量
                shape (B, T_img, n, D_img) ここでnは潜在変数の次元
            media_locations: x内のメディアトークンを識別するブールマスク
                shape (B, T_txt)
            use_cached_media: bool
                Trueの場合、media_locationsで登録された最後のメディア以降のすべてのxを扱う。T_txtはmedia_locations.shape[1]と正確に等しくある必要はない

        戻り値:
            torch.Tensor: ゲート付きクロスアテンションブロックの出力
        """
        logger.info(f"GatedCrossAttentionBlockの順伝播開始 {x.shape=}, {x.dtype=} {media.shape=}, {None if media_locations is None else media_locations.shape=}, {use_cached_media=}")
    
        x = (
            self.attn(
                x,
                media,
                media_locations=media_locations,
                use_cached_media=use_cached_media,
            )
            * self.attn_gate.tanh()
            + x
        )
        x = self.ff(x) * self.ff_gate.tanh() + x

        logger.info(f"GatedCrossAttentionBlockの順伝播完了 {x.shape=} {x.dtype=}")
        return x

### PerceiverAttention

In [ ]:
class PerceiverAttention(nn.Module):
    """
    Perceiver Attentionモジュール
    PerceiverResamplerで使用
    """

    def __init__(self, *, dim, dim_head=64, heads=8):
        logger.info(f"PerceiverAttentionを初期化 {dim=}, {dim_head=}, {heads=}")

        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm_media = nn.LayerNorm(dim)
        self.norm_latents = nn.LayerNorm(dim)

        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

        logger.info("PerceiverAttentionの初期化完了")

    def forward(self, x, latents):
        """
        Perceiver Attentionの順伝播

        引数:
            x (torch.Tensor): 画像特徴量
                shape (b, T, n1, D)
            latents (torch.Tensor): 潜在特徴量
                shape (b, T, n2, D)
                shape (b, T, n2, D)

        戻り値:
            torch.Tensor: Perceiver Attentionの出力
        """
        logger.info(f"PerceiverAttentionの順伝播開始 {x.shape=}, {x.dtype=} {latents.shape=}, {latents.dtype=}")
        x = self.norm_media(x)
        latents = self.norm_latents(latents)

        h = self.heads

        q = self.to_q(latents)
        kv_input = torch.cat((x, latents), dim=-2)
        k, v = self.to_kv(kv_input).chunk(2, dim=-1)
        q, k, v = rearrange_many((q, k, v), "b t n (h d) -> b h t n d", h=h)
        q = q * self.scale

        # attention
        sim = einsum("... i d, ... j d  -> ... i j", q, k)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        out = einsum("... i j, ... j d -> ... i d", attn, v)
        out = rearrange(out, "b h t n d -> b t n (h d)", h=h)
        out = self.to_out(out)
        logger.info(f"PerceiverAttentionの順伝播完了 {out.shape=} {out.dtype=}")
        return out

### PerceiverResampler

In [ ]:
class PerceiverResampler(nn.Module):
    """
    Perceiver Resamplerモジュール
    Flamingoモジュールで使用
    """

    def __init__(
        self,
        *,
        dim,
        depth=6,
        dim_head=64,
        heads=8,
        num_latents=64,
        max_num_media=None,
        max_num_frames=None,
        ff_mult=4,
    ):
        logger.info(f"PerceiverResamplerを初期化 {dim=}, {depth=}, {dim_head=}, {heads=}, {num_latents=}, {max_num_media=}, {max_num_frames=}, {ff_mult=}")

        super().__init__()
        self.latents = nn.Parameter(torch.randn(num_latents, dim))
        self.frame_embs = (
            nn.Parameter(torch.randn(max_num_frames, dim))
            if exists(max_num_frames)
            else None
        )
        self.media_time_embs = (
            nn.Parameter(torch.randn(max_num_media, 1, dim))
            if exists(max_num_media)
            else None
        )

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(
                nn.ModuleList(
                    [
                        PerceiverAttention(dim=dim, dim_head=dim_head, heads=heads),
                        FeedForward(dim=dim, mult=ff_mult),
                    ]
                )
            )

        self.norm = nn.LayerNorm(dim)
        logger.info("PerceiverResamplerの初期化完了")

    def forward(self, x):
        """
        Perceiver Resamplerの順伝播

        引数:
            x (torch.Tensor): 画像特徴量
                shape (b, T, F, v, D)
        戻り値:
            torch.Tensor: Perceiver Resamplerの出力
                shape (b, T, n, D) where n is self.num_latents
        """
        logger.info(f"PerceiverResamplerの順伝播開始 {x.shape=}, {x.dtype=}")

        b, T, F, v = x.shape[:4]

        # frame and media time embeddings
        if exists(self.frame_embs):
            frame_embs = repeat(self.frame_embs[:F], "F d -> b T F v d", b=b, T=T, v=v)
            x = x + frame_embs
        x = rearrange(
            x, "b T F v d -> b T (F v) d"
        )  # flatten the frame and spatial dimensions
        if exists(self.media_time_embs):
            x = x + self.media_time_embs[:T]

        # blocks
        latents = repeat(self.latents, "n d -> b T n d", b=b, T=T)
        for attn, ff in self.layers:
            latents = attn(x, latents) + latents
            latents = ff(latents) + latents

        out = self.norm(latents)
        logger.info(f"PerceiverResamplerの順伝播完了 {out.shape=} {out.dtype=}")
        return out

### Flamingo

In [ ]:
class Flamingo(nn.Module):
    """
    Flamingoモデル
    ビジョンエンコーダーと言語エンコーダーを組み合わせたマルチモーダルモデル
    """

    def __init__(
        self,
        vision_encoder: nn.Module,
        lang_encoder: nn.Module,
        eoc_token_id: int,
        media_token_id: int,
        vis_dim: int,
        cross_attn_every_n_layers: int = 1,
        gradient_checkpointing: bool = False,
    ):
        """
        Flamingoモデルの初期化

        引数:
            vision_encoder (nn.Module): Hugging FaceのCLIPModel
            lang_encoder (nn.Module): Hugging Faceの因果言語モデル
            eoc_token_id (int): <|endofchunk|>のトークンID
            media_token_id (int): <image>のトークンID
            vis_dim (int): ビジュアル特徴量の次元数
                ビジュアル特徴量はこの形状に合わせて最後の次元で投影される
            cross_attn_every_n_layers (int, optional): トランスフォーマーレイヤーの後にクロスアテンションを適用する頻度。デフォルトは1。
            gradient_checkpointing (bool, optional): 勾配チェックポイントを使用するかどうか。デフォルトはFalse。
        戻り値:
            None
        """
        logger.info(f"Flamingoモデルを初期化 {eoc_token_id=}, {media_token_id=}, {vis_dim=}, {cross_attn_every_n_layers=}, {gradient_checkpointing=}")

        super().__init__()
        self.eoc_token_id = eoc_token_id
        self.media_token_id = media_token_id
        self.vis_dim = vis_dim
        if hasattr(lang_encoder.config, "d_model"):
            self.lang_dim = lang_encoder.config.d_model  # mpt uses d_model
        else:
            self.lang_dim = lang_encoder.config.hidden_size

        self.vision_encoder = vision_encoder.visual
        self.perceiver = PerceiverResampler(dim=self.vis_dim)
        self.lang_encoder = lang_encoder
        self.lang_encoder.init_flamingo(
            media_token_id=media_token_id,
            lang_hidden_size=self.lang_dim,
            vis_hidden_size=self.vis_dim,
            cross_attn_every_n_layers=cross_attn_every_n_layers,
            gradient_checkpointing=gradient_checkpointing,
        )
        self._use_gradient_checkpointing = gradient_checkpointing
        self.perceiver._use_gradient_checkpointing = gradient_checkpointing

        logger.info("Flamingoモデルの初期化完了")

    def forward(
        self,
        vision_x: torch.Tensor,
        lang_x: torch.Tensor,
        attention_mask: torch.Tensor = None,
        labels: torch.Tensor = None,
        clear_conditioned_layers: bool = True,
        past_key_values=None,
        use_cache: bool = False,
    ):
        """
        Flamingoの順伝播

        引数:
            vision_x (torch.Tensor): ビジョン入力
                shape (B, T_img, F, C, H, W) with F=1
            lang_x (torch.Tensor): 言語入力ID
                shape (B, T_txt)
            attention_mask (torch.Tensor, optional): アテンションマスク。デフォルトはNone。
            labels (torch.Tensor, optional): ラベル。デフォルトはNone。
            clear_conditioned_layers: Trueの場合、順伝播が完了したら条件付きレイヤーをクリアします。
                同じセットの画像が別の順伝播で再利用される場合は、これをfalseに設定します。
            past_key_values: 事前計算された値を言語モデルに渡します。
                Hugging FaceのCausalLMモデルのpast_key_valuesドキュメントを参照してください。
            use_cache: キャッシュされたキー値を使用するかどうか。Hugging Face CausalLMモデルのuse_cacheドキュメントを参照してください。
        戻り値:
            CausalLMOutputWithPast: 言語モデルの出力
        """
        logger.info(f"Flamingoの順伝播開始 {vision_x.shape=}, {lang_x.shape=}, {None if attention_mask is None else attention_mask.shape=}, {None if labels is None else labels.shape=}, {clear_conditioned_layers=}, {use_cache=}")

        assert (
            self.lang_encoder.initialized_flamingo
        ), "Flamingo layers are not initialized. Please call `init_flamingo` first."

        assert (
            self.lang_encoder._use_cached_vision_x or vision_x is not None
        ), "Must provide either vision_x or have precached media using cache_media()."

        if self.lang_encoder._use_cached_vision_x:
            # Case: use cached; vision_x should be cached and other
            # vision-related inputs should not be provided.
            assert (
                vision_x is None
            ), "Expect vision_x to be None when media has been cached using cache_media(). Try uncache_media() first."
            assert self.lang_encoder.is_conditioned()

        else:
            # Case: do not use caching (i.e. this is a standard forward pass);
            self._encode_vision_x(vision_x=vision_x)
            self._condition_media_locations(input_ids=lang_x)

        output = self.lang_encoder(
            input_ids=lang_x,
            attention_mask=attention_mask,
            labels=labels,
            past_key_values=past_key_values,
            use_cache=use_cache,
        )

        if clear_conditioned_layers:
            self.lang_encoder.clear_conditioned_layers()

        logger.info(f"Flamingoの順伝播完了")

        return output

    def generate(
        self,
        vision_x: torch.Tensor,
        lang_x: torch.Tensor,
        attention_mask: torch.Tensor = None,
        **kwargs,
    ):
        """
        Flamingoのテキスト生成

        引数:
            vision_x (torch.Tensor): ビジョン入力
                shape (B, T_img, F, C, H, W)
                同じチャンク内の画像はT_imgに沿って連結され、フレームはFに沿って連結されます
                現在、F=1のみがサポートされています（単一フレームのビデオ）
            lang_x (torch.Tensor): 言語入力
                shape (B, T_txt)
            **kwargs: Hugging Face CausalLMモデルのgenerateドキュメントを参照してください。注目すべきkwargs:
                max_length (int, optional): 出力の最大長。デフォルトはNone。
                attention_mask (torch.Tensor, optional): アテンションマスク。デフォルトはNone。
                num_beams (int, optional): ビームの数。デフォルトは1。
                max_new_tokens (int, optional): 最大新規トークン数。デフォルトはNone。
                temperature (float, optional): 温度。デフォルトは1.0。
                top_k (int, optional): トップk。デフォルトは50。
                top_p (float, optional): トップp。デフォルトは1.0。
                no_repeat_ngram_size (int, optional): 繰り返し禁止ngramサイズ。デフォルトは0。
                length_penalty (float, optional): 長さペナルティ。デフォルトは1.0。
                num_return_sequences (int, optional): 返されるシーケンスの数。デフォルトは1。
                do_sample (bool, optional): サンプリングを行うかどうか。デフォルトはFalse。
                early_stopping (bool, optional): 早期停止。デフォルトはFalse。
        戻り値:
            torch.Tensor: 生成されたトークンが追加されたlang_x
        """
        logger.info(f"Flamingoのgenerate開始 {vision_x.shape=}, {lang_x.shape=}, {None if attention_mask is None else attention_mask.shape=}, {kwargs.keys()}")
        num_beams = kwargs.pop("num_beams", 1)
        if num_beams > 1:
            vision_x = vision_x.repeat_interleave(num_beams, dim=0)

        self.lang_encoder._use_cached_vision_x = True
        self._encode_vision_x(vision_x=vision_x)

        eos_token_id = kwargs.pop("eos_token_id", self.eoc_token_id)
        output = self.lang_encoder.generate(
            input_ids=lang_x,
            attention_mask=attention_mask,
            eos_token_id=eos_token_id,
            num_beams=num_beams,
            **kwargs,
        )

        self.lang_encoder.clear_conditioned_layers()
        self.lang_encoder._use_cached_vision_x = False
        logger.info(f"Flamingoのgenerate完了 {output.shape=} {output.dtype=}")
        return output

    def _encode_vision_x(self, vision_x: torch.Tensor):
        """
        Compute media tokens from vision input by passing it through vision encoder and conditioning language model.
        Args:
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W)
                Images in the same chunk are collated along T_img, and frames are collated along F
                Currently only F=1 is supported (single-frame videos)

        rearrange code based on https://github.com/dhansmair/flamingo-mini
        """

        assert vision_x.ndim == 6, "vision_x should be of shape (b, T_img, F, C, H, W)"
        b, T, F = vision_x.shape[:3]
        assert F == 1, "Only single frame supported"

        vision_x = rearrange(vision_x, "b T F c h w -> (b T F) c h w")
        with torch.no_grad():
            vision_x = self.vision_encoder(vision_x)[1]
        vision_x = rearrange(vision_x, "(b T F) v d -> b T F v d", b=b, T=T, F=F)
        vision_x = self.perceiver(vision_x)

        for layer in self.lang_encoder._get_decoder_layers():
            layer.condition_vis_x(vision_x)

    def wrap_fsdp(self, wrapper_kwargs, device_id):
        """
        Manually wraps submodules for FSDP and move other parameters to device_id.

        Why manually wrap?
        - all parameters within the FSDP wrapper must have the same requires_grad.
            We have a mix of frozen and unfrozen parameters.
        - model.vision_encoder.visual needs to be individually wrapped or encode_vision_x errors
            See: https://github.com/pytorch/pytorch/issues/82461#issuecomment-1269136344

        The rough wrapping structure is:
        - FlamingoModel
            - FSDP(FSDP(vision_encoder))
            - FSDP(FSDP(perceiver))
            - lang_encoder
                - FSDP(FSDP(input_embeddings))
                - FlamingoLayers
                    - FSDP(FSDP(gated_cross_attn_layer))
                    - FSDP(FSDP(decoder_layer))
                - FSDP(FSDP(output_embeddings))
                - other parameters

        Known issues:
        - Our FSDP strategy is not compatible with tied embeddings. If the LM embeddings are tied,
            train with DDP or set the --freeze_lm_embeddings flag to true.
        - With FSDP + gradient ckpting, one can increase the batch size with seemingly no upper bound.
            Although the training curves look okay, we found that downstream performance dramatically
            degrades if the batch size is unreasonably large (e.g., 100 MMC4 batch size for OPT-125M).

        FAQs about our FSDP wrapping strategy:
        Why double wrap?
        As of torch==2.0.1, FSDP's _post_forward_hook and _post_backward_hook
        only free gathered parameters if the module is NOT FSDP root.

        Why unfreeze the decoder_layers?
        See https://github.com/pytorch/pytorch/issues/95805
        As of torch==2.0.1, FSDP's _post_backward_hook is only registed if the flat param
        requires_grad=True. We need the postback to fire to avoid OOM.
        To effectively freeze the decoder layers, we exclude them from the optimizer.

        What is assumed to be frozen v. unfrozen?
        We assume that the model is being trained under normal Flamingo settings
        with these lines being called in factory.py:
            ```
            # Freeze all parameters
            model.requires_grad_(False)
            assert sum(p.numel() for p in model.parameters() if p.requires_grad) == 0

            # Unfreeze perceiver, gated_cross_attn_layers, and LM input embeddings
            model.perceiver.requires_grad_(True)
            model.lang_encoder.gated_cross_attn_layers.requires_grad_(True)
            [optional] model.lang_encoder.get_input_embeddings().requires_grad_(True)
            ```
        """
        # unfreeze the decoder layers
        for block in self.lang_encoder.old_decoder_blocks:
            block.requires_grad_(True)

        # wrap in FSDP
        with enable_wrap(wrapper_cls=FSDP, **wrapper_kwargs):
            self.perceiver = wrap(wrap(self.perceiver))
            self.lang_encoder.old_decoder_blocks = nn.ModuleList(
                wrap(wrap(block)) for block in self.lang_encoder.old_decoder_blocks
            )
            self.lang_encoder.gated_cross_attn_layers = nn.ModuleList(
                wrap(wrap(layer)) if layer is not None else None
                for layer in self.lang_encoder.gated_cross_attn_layers
            )
            self.lang_encoder.init_flamingo_layers(self._use_gradient_checkpointing)
            self.lang_encoder.set_input_embeddings(
                wrap(wrap(self.lang_encoder.get_input_embeddings()))
            )
            self.lang_encoder.set_output_embeddings(
                wrap(wrap(self.lang_encoder.get_output_embeddings()))
            )
            self.vision_encoder = wrap(wrap(self.vision_encoder))  # frozen

        # manually move non-FSDP managed parameters to device_id
        # these are all in lang_encoder
        apply_with_stopping_condition(
            module=self.lang_encoder,
            apply_fn=lambda m: m.to(device_id),
            apply_condition=lambda m: len(list(m.children())) == 0,
            stopping_condition=lambda m: isinstance(m, FSDP),
        )

        # exclude the original decoder layers from the optimizer
        for block in self.lang_encoder.old_decoder_blocks:
            for p in block.parameters():
                p.exclude_from_optimizer = True

        # set up clip_grad_norm_ function
        def clip_grad_norm_(max_norm):
            self.perceiver.clip_grad_norm_(max_norm)
            for layer in self.lang_encoder.gated_cross_attn_layers:
                if layer is not None:
                    layer.clip_grad_norm_(max_norm)
            self.lang_encoder.get_input_embeddings().clip_grad_norm_(max_norm)

        self.clip_grad_norm_ = clip_grad_norm_

    def _condition_media_locations(self, input_ids: torch.Tensor):
        """
        Compute the media token locations from lang_x and condition the language model on these.
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
        """
        media_locations = input_ids == self.media_token_id

        for layer in self.lang_encoder._get_decoder_layers():
            layer.condition_media_locations(media_locations)

    def cache_media(self, input_ids: torch.Tensor, vision_x: torch.Tensor):
        """
        Pre-cache a prompt/sequence of images / text for log-likelihood evaluations.
        All subsequent calls to forward() will generate attending to the LAST
        image in vision_x.
        This is not meant to be used to cache things for generate().
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W)
                Images in the same chunk are collated along T_img, and frames are collated along F
                Currently only F=1 is supported (single-frame videos)
        """
        self._encode_vision_x(vision_x=vision_x)
        self._condition_media_locations(input_ids=input_ids)
        self.lang_encoder._use_cached_vision_x = True

    def uncache_media(self):
        """
        Clear all conditioning.
        """
        self.lang_encoder.clear_conditioned_layers()
        self.lang_encoder._use_cached_vision_x = False

### FlamingoLayer

In [ ]:
class FlamingoLayer(nn.Module):
    """
    Flamingoレイヤー
    GatedCrossAttentionBlockとDecoderLayerのラッパー
    FlamingoMixinで使用
    """

    def __init__(
        self, gated_cross_attn_layer, decoder_layer, gradient_checkpointing=False
    ):
        logger.info(f"FlamingoLayerを初期化 {gradient_checkpointing=}")

        super().__init__()
        self.gated_cross_attn_layer = gated_cross_attn_layer
        self.decoder_layer = decoder_layer
        self.vis_x = None
        self.media_locations = None
        if self.gated_cross_attn_layer is not None:
            self.gated_cross_attn_layer._use_gradient_checkpointing = (
                gradient_checkpointing
            )
        self.decoder_layer._use_gradient_checkpointing = gradient_checkpointing
        logger.info("FlamingoLayerの初期化完了")

    def is_conditioned(self) -> bool:
        """Check whether the layer is conditioned."""
        return self.vis_x is not None and self.media_locations is not None

    # Used this great idea from this implementation of Flamingo (https://github.com/dhansmair/flamingo-mini/)
    def condition_vis_x(self, vis_x):
        self.vis_x = vis_x

    def condition_media_locations(self, media_locations):
        self.media_locations = media_locations

    def condition_use_cached_media(self, use_cached_media):
        self.use_cached_media = use_cached_media

    def forward(
        self,
        lang_x,
        attention_mask=None,
        **decoder_layer_kwargs,
    ):
        # Cross attention
        if self.gated_cross_attn_layer is not None:
            if self.vis_x is None:
                raise ValueError("vis_x must be conditioned before forward pass")

            if self.media_locations is None:
                raise ValueError(
                    "media_locations must be conditioned before forward pass"
                )

            lang_x = self.gated_cross_attn_layer(
                lang_x,
                self.vis_x,
                media_locations=self.media_locations,
                use_cached_media=self.use_cached_media,
            )

        # Normal decoder layer
        lang_x = self.decoder_layer(
            lang_x, attention_mask=attention_mask, **decoder_layer_kwargs
        )
        return lang_x

### FlamingoLMMixin

In [ ]:
class FlamingoLMMixin(nn.Module):
    """
    言語モデルにクロスアテンションレイヤーを追加するためのミックスイン
    """

    def set_decoder_layers_attr_name(self, decoder_layers_attr_name):
        self.decoder_layers_attr_name = decoder_layers_attr_name

    def _get_decoder_layers(self):
        return getattr_recursive(self, self.decoder_layers_attr_name)

    def _set_decoder_layers(self, value):
        setattr_recursive(self, self.decoder_layers_attr_name, value)

    def init_flamingo(
        self,
        media_token_id,
        lang_hidden_size,
        vis_hidden_size,
        cross_attn_every_n_layers,
        gradient_checkpointing,
    ):
        """
        Initialize Flamingo by adding a new gated cross attn to the decoder. Store the media token id for computing the media locations.
        """
        self.old_decoder_blocks = self._get_decoder_layers()
        self.gated_cross_attn_layers = nn.ModuleList(
            [
                GatedCrossAttentionBlock(
                    dim=lang_hidden_size, dim_visual=vis_hidden_size
                )
                if (layer_idx + 1) % cross_attn_every_n_layers == 0
                else None
                for layer_idx, _ in enumerate(self._get_decoder_layers())
            ]
        )
        self.init_flamingo_layers(gradient_checkpointing)
        self.media_token_id = media_token_id
        self.initialized_flamingo = True
        self._use_cached_vision_x = False

    def init_flamingo_layers(self, gradient_checkpointing):
        """
        Re initializes the FlamingoLayers.
        Propagates any changes made to self.gated_corss_attn_layers or self.old_decoder_blocks
        """
        self._set_decoder_layers(
            nn.ModuleList(
                [
                    FlamingoLayer(
                        gated_cross_attn_layer, decoder_layer, gradient_checkpointing
                    )
                    for gated_cross_attn_layer, decoder_layer in zip(
                        self.gated_cross_attn_layers, self.old_decoder_blocks
                    )
                ]
            )
        )

    def forward(self, input_ids, attention_mask, **kwargs):
        """Condition the Flamingo layers on the media locations before forward()"""
        if not self.initialized_flamingo:
            raise ValueError(
                "Flamingo layers are not initialized. Please call `init_flamingo` first."
            )

        media_locations = input_ids == self.media_token_id

        # if there are media already cached and we're generating and there are no media tokens in the input,
        # we'll assume that ALL input tokens should attend to the last previous media that is cached.
        # this is especially important for HF generate() compatibility, since generate() calls forward()
        # repeatedly one token at a time (with no media tokens).
        # without this check, the model would not attend to any images when generating (after the first token)
        use_cached_media_locations = (
            self._use_cached_vision_x
            and self.is_conditioned()
            and not media_locations.any()
        )

        for layer in self._get_decoder_layers():
            if not use_cached_media_locations:
                layer.condition_media_locations(media_locations)
            layer.condition_use_cached_media(use_cached_media_locations)

        # package arguments for the other parent's forward. since we don't know the order of the arguments,
        # make them all kwargs
        kwargs["input_ids"] = input_ids
        kwargs["attention_mask"] = attention_mask
        return super().forward(**kwargs)  # Call the other parent's forward method

    def is_conditioned(self) -> bool:
        """Check whether all decoder layers are already conditioned."""
        return all(l.is_conditioned() for l in self._get_decoder_layers())

    def clear_conditioned_layers(self):
        for layer in self._get_decoder_layers():
            layer.condition_vis_x(None)
            layer.condition_media_locations(None)
            layer.condition_use_cached_media(None)


### 推論

In [ ]:
clip_vision_encoder_path="ViT-L-14"
clip_vision_encoder_pretrained="openai"
lang_encoder_path="anas-awadalla/mpt-1b-redpajama-200b"
tokenizer_path="anas-awadalla/mpt-1b-redpajama-200b"
cross_attn_every_n_layers=1
cache_dir=None
use_local_files = False
decoder_layers_attr_name: str = None
freeze_lm_embeddings: bool = False

In [ ]:
# 訓練済みのCLIPの視覚エンコーダーと画像プロセッサーを読み込み

vision_encoder, _, image_processor = open_clip.create_model_and_transforms(
    clip_vision_encoder_path,
    pretrained=clip_vision_encoder_pretrained,
    cache_dir=cache_dir,
)

# 視覚エンコーダーが視覚特徴量を出力するように設定
vision_encoder.visual.output_tokens = True

In [ ]:
# テキストトークナイザーを読み込み

text_tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    local_files_only=use_local_files,
    trust_remote_code=True,
    cache_dir=cache_dir,
)

# Flamingの特殊トークンをトークナイザーに追加
text_tokenizer.add_special_tokens(
    {"additional_special_tokens": ["<|endofchunk|>", "<image>"]}
)

if text_tokenizer.pad_token is None:
    # GPT2モデルはパッドトークンを持たない
    # 損失のためにラベルを修正するために使用する
    text_tokenizer.add_special_tokens({"pad_token": "<PAD>"})
    logger.info("パッドトークンを追加")

logger.info(f"トークナイザーを読み込み {text_tokenizer}")

In [ ]:
# 訓練済みの言語エンコーダーを読み込み

lang_encoder = AutoModelForCausalLM.from_pretrained(
    lang_encoder_path,
    local_files_only=use_local_files,
    trust_remote_code=True,
    cache_dir=cache_dir,
)
logger.info(f"テキストエンコーダーを読み込み完了 {lang_encoder=} {lang_encoder.config=}")

In [ ]:
def extend_instance(obj, mixin):
    """Apply mixins to a class instance after creation"""
    base_cls = obj.__class__
    base_cls_name = obj.__class__.__name__
    obj.__class__ = type(
        base_cls_name, (mixin, base_cls), {}
    )  # mixin needs to go first for our forward() logic to work

In [ ]:
if "mpt-1b-redpajama-200b" in lang_encoder_path:

    class EmbeddingFnMixin:
        def get_input_embeddings(self):
            return self.transformer.wte

        def set_input_embeddings(self, new_embeddings):
            self.transformer.wte = new_embeddings

    extend_instance(lang_encoder, EmbeddingFnMixin)
    logger.info("MPT EmbeddingFnMixinを適用")
   
# convert LM to FlamingoLM
extend_instance(lang_encoder, FlamingoLMMixin)
logger.info("FlamingoLMMixinを適用")

In [ ]:
__KNOWN_DECODER_LAYERS_ATTR_NAMES = {
    "opt": "model.decoder.layers",
    "gptj": "transformer.h",
    "gpt-j": "transformer.h",
    "pythia": "gpt_neox.layers",
    "llama": "model.layers",
    "gptneoxforcausallm": "gpt_neox.layers",
    "mpt": "transformer.blocks",
    "mosaicgpt": "transformer.blocks",
}

def _infer_decoder_layers_attr_name(model):
    for k in __KNOWN_DECODER_LAYERS_ATTR_NAMES:
        if k.lower() in model.__class__.__name__.lower():
            return __KNOWN_DECODER_LAYERS_ATTR_NAMES[k]

    raise ValueError(
        f"We require the attribute name for the nn.ModuleList in the decoder storing the transformer block layers. Please supply this string manually."
    )

if decoder_layers_attr_name is None:
    decoder_layers_attr_name = _infer_decoder_layers_attr_name(lang_encoder)
    logger.info(f"デコーダーレイヤーの属性名を推測 {decoder_layers_attr_name=}")

lang_encoder.set_decoder_layers_attr_name(decoder_layers_attr_name)
logger.info(f"デコーダーレイヤーの属性名を設定 {decoder_layers_attr_name=}")

lang_encoder.resize_token_embeddings(len(text_tokenizer))
logger.info(f"テキストエンコーダーのトークン埋め込みをリサイズ {len(text_tokenizer)=}")

In [ ]:
model = Flamingo(
    vision_encoder,
    lang_encoder,
    text_tokenizer.encode("<|endofchunk|>")[-1],
    text_tokenizer.encode("<image>")[-1],
    vis_dim=open_clip.get_model_config(clip_vision_encoder_path)["vision_cfg"][
        "width"
    ],
    cross_attn_every_n_layers=cross_attn_every_n_layers,
    # **flamingo_kwargs,
)

num_params = sum(p.numel() for p in model.parameters())
logger.info(f"Flamingoモデル総パラメータ数: {num_params:,}")

In [ ]:
model.requires_grad_(False)
assert sum(p.numel() for p in model.parameters() if p.requires_grad) == 0
logger.info("モデルの全パラメータをフリーズ")

model.perceiver.requires_grad_(True)
logger.info("Perceiverのパラメータをアンフリーズ")

model.lang_encoder.gated_cross_attn_layers.requires_grad_(True)
logger.info("Gated Cross Attention Layersのパラメータをアンフリーズ")

if not freeze_lm_embeddings:
    model.lang_encoder.get_input_embeddings().requires_grad_(True)
    logger.info("LM Input Embeddingsのパラメータをアンフリーズ")

num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
logger.info(f"訓練可能なパラメータ数: {num_trainable_params:,}")

In [ ]:
# 訓練済みのFlamingoモデルのチェックポイントを読み込み

checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

In [ ]:
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)
demo_image_one

In [ ]:
demo_image_two = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
        stream=True
    ).raw
)
demo_image_two

In [ ]:
query_image = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028352.jpg", 
        stream=True
    ).raw
)
query_image

In [ ]:
# 画像を前処理してテンソルに変換

vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
logger.info(f"画像を前処理 {vision_x[0].shape=}, {vision_x[0].dtype=}")

# (num_media, channels, height, width)
# (3, 3, 224, 224)
vision_x = torch.cat(vision_x, dim=0)
logger.info(f"画像を結合 {vision_x.shape=}, {vision_x.dtype=}")

# (batch_size, num_media, num_frames, channels, height, width)
# (3, 3, 224, 224) -> (1, 3, 1, 3, 224, 224)
vision_x = vision_x.unsqueeze(1).unsqueeze(0)
logger.info(f"画像をリシェイプ {vision_x.shape=}, {vision_x.dtype=}")

In [ ]:
# テキストを前処理

# 生成のためにパディングを左側に設定
text_tokenizer.padding_side = "left"

# <image>と<|endofchunk|>トークンを含むテキストをトークン化
text_list = ["<image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of"]
logger.info(f"テキストを準備 {text_list=}")

lang_x = text_tokenizer(text_list, return_tensors="pt",)
logger.info(f"テキストを前処理 {lang_x['input_ids']=}")

In [ ]:
# テキスト生成
generated_text = model.generate(
    vision_x=vision_x,
    lang_x=lang_x["input_ids"],
    attention_mask=lang_x["attention_mask"],
    max_new_tokens=20,
    num_beams=3,
)
logger.info(f"テキスト生成完了 {generated_text}")

In [ ]:
decoded = text_tokenizer.decode(generated_text[0])
logger.info(f"生成されたテキスト: {decoded}")